In [1]:

from IPython.display import HTML
from IPython.display import display
import os

import numpy as np
import urllib2
import bs4 #this is beautiful soup

from pandas import Series
import pandas as pd
from pandas import DataFrame

from pylab import *
from ipywidgets import widgets
from bs4 import BeautifulSoup
print "stage 1"
import codecs
import re
import urllib

from nltk import word_tokenize
import datetime 
import time 
import collections 
from scipy import stats
import nltk

print "stage 2"

stage 1
stage 2


In [7]:


def calhashid(beds, neighborhood, address, price, source, dt=None):
    
    if (not dt) or (len(dt)!=10):
        dt = datetime.datetime.now()
        dt = str(dt.year%100) + str(dt.month) + str(dt.day) + str(dt.hour) + str(dt.minute)
        
    sid = dt
    
    if beds:
        sid += str(beds).zfill(1)[:1]
    else:
        sid += '0'
    
    nlength = 0
    nparts = 0
    naddress = 0

    if neighborhood:
        neighborhood = neighborhood.strip()
        nlength = len(neighborhood)
        nparts = len(neighborhood.split())
    
    if address:
        address = address.strip()
        naddress = len(address)
        
    temptext='000'
    if nlength > 0: 
        temptext = str(nlength+nparts+naddress).zfill(3)[:3]
        temp=ord(neighborhood[0])
        sid += str(int(temptext)+temp).zfill(3)[:3]
    else:
        sid+=temptext
        
    
    if isinstance(price,float) or (price is None):
        sid+='00'
    else: 
        sid += str(round(price/100.)).zfill(2)[:2]

    sid+=str(source)[:1]
    
    return sid
    

    
    
prices = []
addresses = []
neighborhoods = [] 
imagelinks = []
urllinks = []
beds = []
baths = []
ft2s = []
sourcepages = []
hashids = []

sourcedict = {'http://www.nakedapartments.com/':2, 'http://www.nybits.com/': 3,  'http://streeteasy.com/': 4 }

URL = "http://www.nakedapartments.com/renter/listings/search?nids=3&aids=3%2C11&min_rent=&max_rent=3000&button="
baseURL = re.match(r'\w+://(.+\.\w+)/', URL).group(0)
filepwd="test2.htm"
html=codecs.open(filepwd)
soup = BeautifulSoup(html, "html.parser")
rows = soup.findAll("div", {"class":"listing-row listing-row-standard  mappable-element ab-test mobile-web-style row", })


for row in rows: 
    rowimage = row.find(attrs = {"class": "lazy"})["data-original"]
    imagelinks.append(rowimage)
#     print "imagelink:", rowimage["data-original"]
    rowinfo = row.find(attrs = {"class": "listing-details col-xs-8"})
    listingtitle = rowinfo.find(attrs={"class": "listing-title"})
#     print listingtitle.text
#     print listingtitle.text
    tempprice = re.match(r'\$(\d+),(\d{3})', listingtitle.text)
    prices.append(int(tempprice.group(1)+tempprice.group(2)))
    urllinks.append(listingtitle['href'])
    listingaddress = rowinfo.find(attrs={"class": "listing-address"})
    sizeinfo = rowinfo.find("span", {"class": "listing-size"}).text
    bedsbaths = re.match(r'(\d)BR, (\d)BA', sizeinfo)
    beds.append(int(bedsbaths.group(1)))
    baths.append(int(bedsbaths.group(2)))
    addresses.append(listingaddress.text)
#     print listingaddress.text
    listingneighb = rowinfo.find(attrs = {"class": "listing-neighborhood"})
    neighborhoods.append(listingneighb.text)
    ft2s.append(None)
    sourcepages.append(sourcedict[baseURL]) 
    
    #beds, neighborhood, address, price, source,
    hashids.append(calhashid(beds[-1], neighborhoods[-1], addresses[-1], prices[-1], sourcepages[-1]))
    

URL= "http://streeteasy.com/for-rent/chelsea/price:1000-6000%7Cbeds=1%7Cbaths%3E=1%7Cno_fee:1"
baseURL = re.match(r'\w+://(.+\.\w+)/', URL).group(0)
filepwd="test4.htm"
html=codecs.open(filepwd)
soup = BeautifulSoup(html, "html.parser")
rows = soup.findAll("div", {"class":'item'})

print len(rows)
for row in rows: 
    urllinks.append(baseURL+row.find("div", {"class": "details-title"}).find("a")["href"])
    tempprice = re.match(r'\$(\d+),(\d{3})', row.find("span", {"class": "price"}).text)
    prices.append(int(tempprice.group(1)+tempprice.group(2)))
#     prices.append(row.find("span", {"class": "price"}).text)
    imagelinks.append(row.find(attrs = {"class": "lazy"})["data-original"])
    addresses.append(row.find(attrs = {"class": "details-title"}).find("a").text.split(' #')[0])
    details = row.findAll(attrs = {"class":  'details_info'})
    sizeinfo = details[0].text
    neighbtext = re.sub('\s+', ' ', details[1].text)
    neighb = re.match(r'.+Rental Unit in (.*)', neighbtext)
    if neighb: 
        temp = neighb.groups(1)[0]        
        neighborhoods.append(temp)
    else:
        neighborhoods.append('')

    
    snums= re.findall(r'(\d+)', sizeinfo)
    n = len(snums)
    if n>=1:
        beds.append(int(snums[0]))
        if n>=2:
            baths.append(int(snums[1]))
            if n>2:
                ft2s.append(int(snums[2]))
            else: 
                ft2s.append(None)
        else:
            baths.append(None)
    else: 
        beds.append(None)
        
    sourcepages.append(sourcedict[baseURL]) 
    hashids.append(calhashid(beds[-1], neighborhoods[-1], addresses[-1], prices[-1], sourcepages[-1]))
        
                
URL= "http://www.nybits.com/search/?_a%21process=y&_rid_=3&_ust_todo_=65733&_xid_=B8ek6K2lRUbcHi-1467834565&%21%21atypes=2br&%21%21rmin=1000&%21%21rmax=5000&%21%21fee=nofee&%21%21orderby=dateposted&submit=+SHOW+RENTAL+APARTMENTS+&%21%21nsearch=midtown_south&%21%21nsearch=sutton_place&%21%21nsearch=lincoln_square&%21%21nsearch=inwood"
baseURL = re.match(r'\w+://(.+\.\w+)/', URL).group(0)
filepwd="test3.htm"
html=codecs.open(filepwd)
soup = BeautifulSoup(html, "html.parser")
table = soup.find("table", {"id": "rentaltable"})
rows = table.findAll("tr")

print "The number of rows in nybits: ", len(rows) 


tempaddresses = []
tempimagelinks = []
tempprices = []
tempbeds = []
tempneighborhood = []
tempbedroom = []
tempurllinks = []

n = 3
i=0
def call_agent(url):
    headers = { 'User-Agent' : 'Mozilla/5.0' }
    req = urllib2.Request(url, None, headers)
    return urllib2.urlopen(req).read()    

    
for row in rows:
    if i>=n: 
        break
    firtcolumn = row.find("td", {"class": "lst_sr_price"})
    if (not firtcolumn) or (len(firtcolumn.text)==0): 
        continue
    secondcolumn = row.find("td", {"class": "lst_sr_topcell"})
    if not secondcolumn:
        continue
    featureimage = secondcolumn.find("img")
#     print featureimage
    if featureimage: 
        i+=1
        url = secondcolumn.find("a")["href"]
        tempurllinks.append(url)
        pricetext =  firtcolumn.text.replace('\n', '')
        print "this is the first column: ", pricetext
        pricestring = re.match(r'.*\$(\d+),(\d{3})', pricetext) 
        tempprices.append(int(pricestring.group(1)+pricestring.group(2)))
        html = call_agent(url)
        soup = BeautifulSoup(html, "html.parser")
        title = soup.find("div", {"id": "dancefloor"}).find('h1').text
        bedroom = re.match(r'(\d).Bedroom', title).group(1)
        tempbedroom.append(int(bedroom))



        content = soup.find("div", {"class": "listing_content"})
        tempimagelinks.append(content.find("img")["src"])
        table = soup.find("table", {"class": "listing_summary"})
        rows = table.findAll("tr")
        for row in rows: 
            columns = row.findAll("td")
            if "Building" in columns[0].text:
                tempaddresses.append(columns[1].text.replace('\n', ''))
                continue
            elif "Neighborhood" in columns[0].text:
                tempneighb = columns[1].text.replace('\n', '')
                neigb = tempneighb.split(';')[-1]
                tempneighborhood.append(neigb)
        
        hashids.append(calhashid(tempbedroom[-1], tempneighborhood[-1], tempaddresses[-1], tempprices[-1], 3))
                
        

print tempaddresses, tempneighborhood, tempbedroom, tempprices

prices += tempprices
addresses += tempaddresses
neighborhoods += tempneighborhood
imagelinks += tempimagelinks
urllinks += tempurllinks
beds += tempbedroom

for i in range(len(tempprices)):
    baths.append(None)
    ft2s.append(None)
    sourcepages.append(3)
    


print len(addresses), len(beds), len(prices), len(baths), len(imagelinks), len(neighborhoods), len(urllinks), len(ft2s), len(sourcepages)
finaldict = {'hashid': hashids,'full_address': addresses, 'bedroom':beds, 'price': prices, 'bathroom': baths, 'image_url': imagelinks, 'neighborhood': neighborhoods, 'linkurl': urllinks, 'source': sourcepages}
                  
df = pd.DataFrame(finaldict)
print df                


14
The number of rows in nybits:  31
this is the first column:  $4,895
this is the first column:  $2,995
this is the first column:  $4,800
[u'The Epic, New York, NY, 10001', u'63 West, New York, NY, 10023', u'140 East 56th Street, New York, NY, 10022'] [u'Midtown South, Manhattan', u'Upper West Side, Manhattan', u'Sutton Place, Manhattan'] [2, 2, 2] [4895, 2995, 4800]
26 26 26 26 26 26 26 26 26
    bathroom  bedroom                               full_address  \
0        1.0        1                               West 77th St   
1        1.0        1                              West 104th St   
2        1.0        1                              West 105th St   
3        1.0        1                                 West 105th   
4        1.0        1                              West 106th St   
5        1.0        1                                  West 78th   
6        1.0        1                              West 106th St   
7        1.0        1                                  Wes

In [5]:


# def call_agent(url):
#     headers = { 'User-Agent' : 'Mozilla/5.0' }
#     req = urllib2.Request(url, None, headers)
#     return urllib2.urlopen(req).read()    


# url = "https://www.zumper.com/apartments-for-rent/new-york-ny/upper-east-side/2-beds/price-1400,4400?box=-73.95970,40.75866,-73.94318,40.78020&bathrooms=2&property-categories=apartment"
# html = call_agent(url)

# with open('content2.html', 'w') as fp:
#     fp.write(html)
#     fp.close()


filepwd="content.html"
html=codecs.open(filepwd)
soup = BeautifulSoup(html, "html.parser")
print soup.prettify()


# from selenium import webdriver
# import re

# driver = webdriver.PhantomJS() # or add to your PATH
# driver.set_window_size(1024, 768) # optional
# driver.get('https://www.zumper.com/apartments-for-rent/new-york-ny/no-fee/1-beds/under-2900')
# driver.save_screenshot('screen.png') # save a screenshot to disk
    
    
    

# print len(addresses), len(beds), len(prices), len(baths), len(imagelinks), len(neighborhoods), len(urllinks)
# finaldict = {'full_address': addresses, 'bedroom':beds, 'price': prices, 'bathroom': baths, 'image_url': imagelinks, 'neighborhood': neighborhoods, 'linkurl': urllinks}
                  
# df = pd.DataFrame(finaldict)
# print df                


    

<!DOCTYPE html>
<html lang="en">
 <head>
  <script>
   window.ts_endpoint_url = "https:\/\/slack.com\/beacon\/timing";

(function(e) {
	var n=Date.now?Date.now():+new Date,r=e.performance||{},t=[],a={},i=function(e,n){for(var r=0,a=t.length,i=[];a>r;r++)t[r][e]==n&&i.push(t[r]);return i},o=function(e,n){for(var r,a=t.length;a--;)r=t[a],r.entryType!=e||void 0!==n&&r.name!=n||t.splice(a,1)};r.now||(r.now=r.webkitNow||r.mozNow||r.msNow||function(){return(Date.now?Date.now():+new Date)-n}),r.mark||(r.mark=r.webkitMark||function(e){var n={name:e,entryType:"mark",startTime:r.now(),duration:0};t.push(n),a[e]=n}),r.measure||(r.measure=r.webkitMeasure||function(e,n,r){n=a[n].startTime,r=a[r].startTime,t.push({name:e,entryType:"measure",startTime:n,duration:r-n})}),r.getEntriesByType||(r.getEntriesByType=r.webkitGetEntriesByType||function(e){return i("entryType",e)}),r.getEntriesByName||(r.getEntriesByName=r.webkitGetEntriesByName||function(e){return i("name",e)}),r.clearMarks||(r.clearMarks=r.w

<


In [2]:
# nltk.download()

def call_agent(url):
    headers = { 'User-Agent' : 'Mozilla/5.0' }
    req = urllib2.Request(url, None, headers)
    return urllib2.urlopen(req).read()    

def save_html(url, filename, latesthtmlfilename): 
    html = call_agent(url)
    soup = BeautifulSoup(html, "lxml")
    rows = soup.find("p", {"class":"row"})
    
    if rows:
        lasttime=rows.time['datetime']
        newfilename='%s_%s.html'%(filename,lasttime)
        if newfilename>latesthtmlfilename:
            with open('Files/%s_%s.html'%(filename,lasttime), 'w') as fp: 
                fp.write(html)
                fp.close()
            return True
        else: 
            print "The earlier html files than %s already exists!"%lasttime
            return False
    
    else: 
        print "No content for this page!!"
        return False
    
def save_pages(): 
    urls=[]
    urls.append("http://newyork.craigslist.org/search/abo?sort=date&query=my%20apartment")
    urls.append("http://newyork.craigslist.org/search/roo?sort=date&query=my%20apartment")
    filenames=["abo_myapartment","roo_myapartment"]
    
    n=len(urls)
    j=0
    for i in range(n):
        f=filenames[i]
        htmlnames=[]
        for file in os.listdir("Files"):
            if file.startswith(f[:3]) and file.endswith(".html"):
                htmlnames.append(file)

        htmlnames.sort()
        latesthtmlfilename=htmlnames[-1]
    
        url=urls[i]
        while j<1000:
            sleeptime=4+4*np.random.random()
            print "I am sleeping for %f seconds"%sleeptime
            time.sleep(sleeptime)
            url2=url+"&s="+str(j)
            filename=filenames[i]
            didit=save_html(url2, filename, latesthtmlfilename)
            if not didit:
                break;
            j+=100


save_pages()



I am sleeping for 5.504372 seconds
I am sleeping for 7.265077 seconds
I am sleeping for 7.389717 seconds
The earlier html files than 2016-06-13 19:05 already exists!
I am sleeping for 7.927196 seconds
I am sleeping for 5.627328 seconds
I am sleeping for 5.248326 seconds
I am sleeping for 5.617581 seconds
I am sleeping for 5.128154 seconds
I am sleeping for 7.639993 seconds
I am sleeping for 4.739344 seconds
The earlier html files than 2016-06-13 16:39 already exists!


In [5]:

def find_bedrooms_from_title(title):
    roomterms=["br", "rooms", "bedrooms", "bdrm"]
    title=title.lower()
    if "studio" in title: 
        output="studio"
        return output
    
    temp=re.search('\d+\s?br', title)
    if temp: 
        output=temp.group(0).replace(' ', '')
        return output
    else: 
        temp=re.search('\d+\s?bdrm', title)
        if temp: 
            output=temp.group(0)
        else: 
            title=title.lower()
            temp=re.search('\d+\s?bedroom', title)
            if temp: 
                output= temp.group(0)
            else:
                #print "There is no bedroom in: ", title
                output=None
    
    if "loft" in title: 
        if output:
            nbeds=re.search('\d+', output).group(0) 
            return nbeds+"br loft"
        else: 
            return "loft"
    else: 
        if output: 
            nbeds=re.search('\d+', output).group(0) 
            return nbeds+"br"
        else: 
            return None

def find_apt_or_room(title):
    title=title.lower()
    terms=[" room ", " bedroom ", "roommate" ]
    for t in terms: 
        if t in title: 
            return "room"
        else: 
            return "apt"
    
def findlasttime(currenttime, ndays):
    year=currenttime.year
    month=currenttime.month
    day=currenttime.day
    
    ndays=min(ndays,7)
    if day>ndays:
        return "%s-%s-%s"%(year, str(month).zfill(2), str(day-ndays).zfill(2))
    else:
        return "%s-%s-%s"%(year, str(month-1).zfill(2), str(day-ndays+30).zfill(2))


def create_csv():
    htmlnames=[]
    csvnames=[]
    for file in os.listdir("Files"):
        if file.endswith(".html"):
            htmlnames.append(file)
    #save_html(url, filename)
    for file in os.listdir("Files"):
        if file.endswith(".csv"):
            csvnames.append(file[:-4])
    
    htmlnames=[h for h in htmlnames if h[:-4] not in csvnames]
    
    sectiondict={"abo": "apt by owner", "roo": "rooms & shares"}
    for filename in htmlnames:
        filepwd="Files/%s"%filename
        html=codecs.open(filepwd)
        soup = BeautifulSoup(html, "lxml")
        rows = soup.findAll("p", {"class":"row"})
        hreflinks=[]
        titles=[]
        prices=[]
        datetimes=[]
        neighborhoods=[]
        beds=[]
        aptorroom=[]
        dublicate=[]
        owner=[]
        duplicate=[]
        duplicatetime=[]
        broker=[]
        section=[]

        for r in rows:
            title=r.find("span", {"id":"titletextonly"}).text
            titles.append(title)
            beds.append(find_bedrooms_from_title(title))
            aptorroom.append(find_apt_or_room(title))
            hreflinks.append("http://newyork.craigslist.org"+r.a["href"])
            datetimes.append(r.time["datetime"]) 
            reneighb = re.search(r"\(.*\)", r.find("span", {"class":"pnr"}).text)
            reprice=r.find("span", {"class":"price"})
            if reprice:
                prices.append(int(reprice.text[1:]))
            else: 
                prices.append(None)

            if reneighb:
                neighborhoods.append(reneighb.group(0)[1:-1])
            else:
                neighborhoods.append(None)
            duplicate.append(0)
            duplicatetime.append(0)
            broker.append(0)
            section.append(sectiondict[filename[:3]])



        pagedict={"title":titles, "datetime": datetimes, "type": aptorroom, "beds": beds, "price":prices, "neighborhood": neighborhoods, "broker":broker,"href": hreflinks, "duplicate": duplicate, "duplicatetime": duplicatetime, "section": section}
        df=pd.DataFrame(pagedict)
        print filename
        df.to_csv("Files/%s.csv"%filename[:-4], sep=',', encoding='utf-8')    
        

create_csv()




abo_myapartment_2016-06-14 15:43.html
abo_myapartment_2016-06-15 13:36.html
roo_myapartment_2016-06-13 22:39.html
roo_myapartment_2016-06-14 07:16.html
roo_myapartment_2016-06-14 12:32.html
roo_myapartment_2016-06-14 14:23.html
roo_myapartment_2016-06-14 17:56.html
roo_myapartment_2016-06-14 23:23.html


In [6]:


#websites=['streeteasy':0 , 'nybits': 1, 'nakedapartments':3, 'hotpads':4, 'craigslist':5]


def cal_simil(item1, item2):
    nit1=len(item1)
    minlength=min(len(item1), len(item2))
    if minlength==0: 
        return 0

    elif minlength==nit1: 
        shorter=item1
        longer=item2
    else: 
        shorter=item2
        longer=item1

    nwords=0
    for word in shorter: 
        if word in longer: 
            nwords+=1
    return float(nwords)/len(longer)
                
        
def add_duplicatecolumn(df): 
    duplicate=[]
    duplicatetime=[]
    pricelist=list(df['price'])
    titlelist=list(df['title'])
    nonasciititles=[re.sub(r'[^\x00-\x7F]+',' ', text) for text in titlelist]
    titlelist_token=[word_tokenize(t) for t in nonasciititles if len(t)>0]
    neighborlist=list(df['neighborhood'])
    neighborlist_token=[word_tokenize(str(n)) for n in neighborlist if len(str(n))>0]
    datelist=list(df['datetime'])
    N=len(pricelist)
    for i in range(N): 
#         print i
        neighb=neighborlist_token[i]
        price=pricelist[i]
        title=titlelist_token[i]
        for j in range(i+1, N): 
            newprice=pricelist[j]
            newtitle=titlelist_token[j]
            newneighb=neighborlist_token[j]
            d2=cal_simil(neighb, newneighb)
            if d2>0.8:
                d1=cal_simil(title, newtitle)
                if newprice==price:
                    d3=1
                    similarity=sum([d1, d2, d3])/3.
                else: 
                    similarity=sum([d1, d2])/2.
                    
                if similarity>0.75: 
                    duplicate.append(similarity)
                    duplicatetime.append(datelist[j])
                    break;
                    
            if j==N-1: 
                    duplicate.append(0)
                    duplicatetime.append(None)
                    
        if i==N-1:
            duplicate.append(0)
            duplicatetime.append(None)


    df=df.copy()
    print df.shape[0], len(duplicate)
    df['duplicate']=duplicate
    df['duplicatetime']=duplicatetime
    return df

def add_broker(df):
    duplicate=[]
    duplicatetime=[]
    titlelist=list(df['title'])
    nonasciititles=[re.sub(r'[^\x00-\x7F]+',' ', text) for text in titlelist]
    
    alltitles=word_tokenize(' '.join(nonasciititles).lower())

    brokerfactor=[]
    titlerepetitionfactor=[]
    titlekeywordfactor=[]
    titlecapitalized=[]
    allwords=[]
    
    allwords=word_tokenize(' '.join(nonasciititles))
    wordcount=collections.Counter(allwords)
    
    for s in nonasciititles: 
        cl=sum([1 for c in s if c.isupper()])
        sl=len(s)
        titlecapitalized.append(float(cl)/sl)
        words=word_tokenize(str(s))
        titlerepetitionfactor.append(sum([wordcount[w] for w in words]))

    maxcount=float(max(titlerepetitionfactor))
    repetitionpercentile=[stats.percentileofscore(titlerepetitionfactor, a, 'strict') for a in titlerepetitionfactor]
    brokerscore=[int((repetitionpercentile[i]+100*titlecapitalized[i])/2) for i in range(len(repetitionpercentile))]
    filteredbrokerscore=[]
    for x in brokerscore: 
        if x>=40:
            filteredbrokerscore.append(x)
        else: 
            filteredbrokerscore.append(0)
            
    df=df.copy()
    df['broker']=filteredbrokerscore
#     figure()
#     hist(filteredbrokerscore)
#     show()
    return df

    
        
def aggregate_csv(): 
    csvnames=[]
    resultlistsofdf=[]
    for file in os.listdir("Files"):
        if file.endswith(".csv"):
            csvnames.append(file)
    csvnames.sort(reverse=True) 
    sections=[]
    for name in csvnames: 
        sections.append(name[:3])
    
    sectionsset=list(set(sections))
    sectionsset.remove('fin')
    for sec in sectionsset: 
        tempnames=[name for name in csvnames if name.startswith(sec)]#.sort(reverse=True)
        tempdfs=[]
        previousdatetimelist=[]
        for name in tempnames: 
            newdf=pd.read_csv("Files/"+name)[["datetime","type", "beds", "price", "neighborhood", "title", "href", "duplicate", "duplicatetime", "broker", "section"]]
            if len(previousdatetimelist)>0:
                mindate=min(previousdatetimelist)
#                 print name, mindate
                newdf=newdf[newdf['datetime']<mindate]
                if newdf.shape[0]==0:
                    continue;

            tempdfs.append(newdf)            
            previousdatetimelist=newdf['datetime']
#             print "the new df shape is: " , newdf.shape[0]

        tempdf=pd.concat(tempdfs).sort_values(by=["datetime"], ascending=0)
#         print "the tempdf size is:", tempdf.shape[0]
        resultlistsofdf.append(tempdf)
        
#     print "the result list of df length is: ", len(resultlistsofdf)
    df=pd.concat(resultlistsofdf).sort_values(by=["datetime"], ascending=0)
#     print "The final df size is: ", df.shape[0]
    n=df.shape[0]
    df=df.set_index([range(n)])
    df=add_broker(df)
    df=add_duplicatecolumn(df)
    pd.set_option('display.max_colwidth', -1)
    
#     with open("Files/finaltable.html", 'w') as fp:
#         fp.write(df.to_html().encode('utf8'))
#         fp.close()
    df[:200].reset_index().to_json("Files/finalCLListings.json", orient='records'); 
    N=df.shape[0]
    df[:min(5000, N)].to_csv("Files/finalcsv.csv", sep=',', encoding='utf-8')    
            
    
def search_pages():
    filenames=["abo_myapartment","roo_myapartment"]
    craigsection=["apt by owner", "rooms and shares "]
    n=len(urls)
    csvnames=[]
    for file in os.listdir("Files"):
        if file.endswith(".csv"):
            csvnames.append(file)
    
    csvnames.sort()
    resultlistsofdf=[]
#     if csvnames:
#         latestdf=pd.read_csv("Files/"+csvnames[-1])[["datetime","type", "beds", "price", "neighborhood", "title", "href", "duplicate", "duplicatetime", "broker"]]
#         timelist=list(latestdf['datetime'])
#         timelist.sort()
#         lasttime=timelist[-1]
#         resultlistsofdf.append(latestdf)
    
#     else: 
#         currenttime=datetime.datetime.now()
#         lasttime=findlasttime(currenttime, 7)
#     for i in range(n):
#         dfs=[]
#         url=urls[i]
#         f=filenames[i]
#         j=0
        
#         print lasttime
#         resultlistsofdf.append(result[["datetime","type", "beds", "price", "neighborhood", "title", "href", "duplicate", "duplicatetime", "broker"]])
                
#             j+=100
    
#     print len(resultlistsofdf)

#     df=pd.concat(resultlistsofdf).sort(["datetime"], ascending=0)
#     timelist=list(df["datetime"])
#     timelist.sort()
#     lasttime=timelist[-1]
#     fn='Files/listings_%s.csv'%lasttime.replace(' ','_').replace(":","-")
#     df.to_csv(fn, sep=',', encoding='utf-8')    
        
        
# finalresult=finalresult.sort(["datetime"], ascending=0)
# finalresult.to_csv('Files/AptList.csv', sep=',', encoding='utf-8')

# previousresults=pd.read_csv('Files/previous_list.csv')

#search_pages()
aggregate_csv()

6097 6097
